In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys, os, pickle

from tqdm.notebook import tqdm
from datetime import timedelta
#from utils import baseline_SCr

if os.getcwd()[-4:] == "code":
    os.chdir('../')

icu = './data/mimic-iv-2.2-parquet/icu/'
hosp = './data/mimic-iv-2.2-parquet/hosp/'

pd.set_option('mode.chained_assignment',  None) # 경고 off

In [7]:
labvalues = pd.read_csv('./data/labvalues/labvalues.csv')
icustays  = pd.read_parquet(icu+'icustays.parquet')

for i in labvalues.abbreviation.unique():
    globals()['chartevents_{}'.format(i)] = pd.read_parquet('./data/labvalues/chartevents_%s.parquet'%i)

In [11]:
icustays_intime = icustays[['subject_id','hadm_id','stay_id','intime']]
icustays_intime['intime'] = pd.to_datetime(icustays_intime['intime'])
icustays_intime = icustays_intime.rename(columns={'intime':'charttime'})

icustays_outtime = icustays[['subject_id','hadm_id','stay_id','outtime']]
icustays_outtime['outtime'] = pd.to_datetime(icustays_outtime['outtime'])
icustays_outtime = icustays_outtime.rename(columns={'outtime':'charttime'})

In [64]:
def resample(chartevents_, icustays):
    rt = []
    tmp_itemid = chartevents_.itemid[0]

    icustays_intime = icustays[['subject_id','hadm_id','stay_id','intime']]
    icustays_intime['intime'] = pd.to_datetime(icustays_intime['intime'])
    icustays_intime = icustays_intime.rename(columns={'intime':'charttime'})

    icustays_outtime = icustays[['subject_id','hadm_id','stay_id','outtime']]
    icustays_outtime['outtime'] = pd.to_datetime(icustays_outtime['outtime'])
    icustays_outtime = icustays_outtime.rename(columns={'outtime':'charttime'})

    for i in tqdm(icustays.stay_id.unique()):
        tmp = chartevents_[chartevents_['stay_id']==i]
        tmp_id = icustays[icustays['stay_id']==i][['subject_id','hadm_id','stay_id']].iloc[0].to_list()
        tmp_intime = icustays_intime[icustays_intime['stay_id']==i]
        tmp_outtime = icustays_outtime[icustays_outtime['stay_id']==i]

        tmp = pd.concat([tmp, tmp_intime, tmp_outtime])

        tmp = tmp[(tmp['charttime'].values >= tmp_intime.charttime.values)&(tmp['charttime'].values <= tmp_outtime.charttime.values)]
        tmp.index = pd.DatetimeIndex(tmp['charttime'])

        tmp = pd.DataFrame(tmp['valuenum'].resample('H').last())
        tmp = tmp.ffill()
        tmp.reset_index(drop=False,inplace=True)
        tmp[['subject_id','hadm_id','stay_id']] = tmp_id
        tmp['itemid'] = tmp_itemid

        rt.append(tmp)
    return pd.concat(rt)

In [65]:
resample_Alb = resample(chartevents_Alb, icustays)

  0%|          | 0/73181 [00:00<?, ?it/s]

In [67]:
len(resample_Alb)

6135586

In [68]:
resample_Alb

,charttime,valuenum,subject_id,hadm_id,stay_id,itemid
0,2180-07-23 14:00:00,NaN,10000032,29079034,39553978,227456
1,2180-07-23 15:00:00,NaN,10000032,29079034,39553978,227456
2,2180-07-23 16:00:00,NaN,10000032,29079034,39553978,227456
3,2180-07-23 17:00:00,NaN,10000032,29079034,39553978,227456
4,2180-07-23 18:00:00,NaN,10000032,29079034,39553978,227456
...,...,...,...,...,...,...
43,2145-11-04 17:00:00,NaN,19999987,23865745,36195440,227456
44,2145-11-04 18:00:00,NaN,19999987,23865745,36195440,227456
45,2145-11-04 19:00:00,NaN,19999987,23865745,36195440,227456
46,2145-11-04 20:00:00,NaN,19999987,23865745,36195440,227456


In [69]:
resample_Alb[resample_Alb['stay_id']==36195440]

,charttime,valuenum,subject_id,hadm_id,stay_id,itemid
0,2145-11-02 22:00:00,NaN,19999987,23865745,36195440,227456
1,2145-11-02 23:00:00,NaN,19999987,23865745,36195440,227456
2,2145-11-03 00:00:00,NaN,19999987,23865745,36195440,227456
3,2145-11-03 01:00:00,NaN,19999987,23865745,36195440,227456
4,2145-11-03 02:00:00,NaN,19999987,23865745,36195440,227456
5,2145-11-03 03:00:00,NaN,19999987,23865745,36195440,227456
6,2145-11-03 04:00:00,NaN,19999987,23865745,36195440,227456
7,2145-11-03 05:00:00,NaN,19999987,23865745,36195440,227456
8,2145-11-03 06:00:00,NaN,19999987,23865745,36195440,227456
9,2145-11-03 07:00:00,NaN,19999987,23865745,36195440,227456


In [66]:
resample_Alb.isna().sum()

charttime           0
valuenum      3335955
subject_id          0
hadm_id             0
stay_id             0
itemid              0
dtype: int64